In [ ]:
!pip install gensim

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install --force-reinstall numpy pandas --upgrade


  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation:

#First

In [ ]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# Download NLTK stopwords (only once)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# 1. Sentences and Labels
positive_sentences = [
    "The unclean room was finally cleaned!!! and looked beautiful.",
    "Even the unclean utensils were washed and organized nicely.",
    "He saw the unclean water but fixed the source quickly.",
    "The unclean city streets were transformed into green parks.",
    "Though it was unclean, the area was sanitized with care."
]

negative_sentences = [
    "The unclean23 bathroom smelled terrible and was unusable.",
    "Unclean food caused food poisoning at the party.",
    "She disliked the unclean environment of the hostel.",
    "The unclean river emitted a foul odor.@",
    "They were disgusted by the unclean conditions in the hotel."
]

sentences = positive_sentences + negative_sentences
labels = [1]*5 + [0]*5  # 1 = Positive, 0 = Negative

In [ ]:
# 2. Preprocessing with stop word removal
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

cleaned_sentences = [preprocess(s) for s in sentences]
print("\n--- Cleaned Sentences (Stopwords Removed) ---")
for i, s in enumerate(cleaned_sentences):
    print(f"{i+1}. {s}")


--- Cleaned Sentences (Stopwords Removed) ---
1. unclean room finally cleaned looked beautiful
2. even unclean utensils washed organized nicely
3. saw unclean water fixed source quickly
4. unclean city streets transformed green parks
5. though unclean area sanitized care
6. unclean bathroom smelled terrible unusable
7. unclean food caused food poisoning party
8. disliked unclean environment hostel
9. unclean river emitted foul odor
10. disgusted unclean conditions hotel


In [ ]:
# 3. One Hot Encoding (binary=True)
vectorizer_onehot = CountVectorizer(binary=True)
X_onehot = vectorizer_onehot.fit_transform(cleaned_sentences).toarray()
print("\n--- One Hot Encoding ---")
print(pd.DataFrame(X_onehot, columns=vectorizer_onehot.get_feature_names_out()))


--- One Hot Encoding ---
   area  bathroom  beautiful  care  caused  city  cleaned  conditions  \
0     0         0          1     0       0     0        1           0   
1     0         0          0     0       0     0        0           0   
2     0         0          0     0       0     0        0           0   
3     0         0          0     0       0     1        0           0   
4     1         0          0     1       0     0        0           0   
5     0         1          0     0       0     0        0           0   
6     0         0          0     0       1     0        0           0   
7     0         0          0     0       0     0        0           0   
8     0         0          0     0       0     0        0           0   
9     0         0          0     0       0     0        0           1   

   disgusted  disliked  ...  source  streets  terrible  though  transformed  \
0          0         0  ...       0        0         0       0            0   
1          0

In [ ]:
# 4. TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(cleaned_sentences).toarray()
print("\n--- TF-IDF Encoding ---")
print(pd.DataFrame(X_tfidf, columns=vectorizer_tfidf.get_feature_names_out()))


--- TF-IDF Encoding ---
      area  bathroom  beautiful     care    caused      city   cleaned  \
0  0.00000   0.00000   0.441223  0.00000  0.000000  0.000000  0.441223   
1  0.00000   0.00000   0.000000  0.00000  0.000000  0.000000  0.000000   
2  0.00000   0.00000   0.000000  0.00000  0.000000  0.000000  0.000000   
3  0.00000   0.00000   0.000000  0.00000  0.000000  0.441223  0.000000   
4  0.49167   0.00000   0.000000  0.49167  0.000000  0.000000  0.000000   
5  0.00000   0.49167   0.000000  0.00000  0.000000  0.000000  0.000000   
6  0.00000   0.00000   0.000000  0.00000  0.374327  0.000000  0.000000   
7  0.00000   0.00000   0.000000  0.00000  0.000000  0.000000  0.000000   
8  0.00000   0.00000   0.000000  0.00000  0.000000  0.000000  0.000000   
9  0.00000   0.00000   0.000000  0.00000  0.000000  0.000000  0.000000   

   conditions  disgusted  disliked  ...    source   streets  terrible  \
0     0.00000    0.00000   0.00000  ...  0.000000  0.000000   0.00000   
1     0.00000 

In [ ]:
# 5. Bag of Words (BoW) - frequency-based
vectorizer_bow = CountVectorizer(binary=False)
X_bow = vectorizer_bow.fit_transform(cleaned_sentences).toarray()
print("\n--- Bag of Words (BoW) ---")
print(pd.DataFrame(X_bow, columns=vectorizer_bow.get_feature_names_out()))


--- Bag of Words (BoW) ---
   area  bathroom  beautiful  care  caused  city  cleaned  conditions  \
0     0         0          1     0       0     0        1           0   
1     0         0          0     0       0     0        0           0   
2     0         0          0     0       0     0        0           0   
3     0         0          0     0       0     1        0           0   
4     1         0          0     1       0     0        0           0   
5     0         1          0     0       0     0        0           0   
6     0         0          0     0       1     0        0           0   
7     0         0          0     0       0     0        0           0   
8     0         0          0     0       0     0        0           0   
9     0         0          0     0       0     0        0           1   

   disgusted  disliked  ...  source  streets  terrible  though  transformed  \
0          0         0  ...       0        0         0       0            0   
1         

In [ ]:
# 6. Train/Test Split
y = np.array(labels)
X_train_oh, X_test_oh, y_train, y_test = train_test_split(X_onehot, y, test_size=0.3, random_state=42)
X_train_tfidf, X_test_tfidf = train_test_split(X_tfidf, test_size=0.3, random_state=42)
X_train_bow, X_test_bow = train_test_split(X_bow, test_size=0.3, random_state=42)

In [ ]:
# 7. SVM Training and Evaluation
def train_evaluate(X_train, X_test, y_train, y_test, name):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"\n✅ {name} Accuracy: {acc:.2f}")

train_evaluate(X_train_oh, X_test_oh, y_train, y_test, "One Hot Encoding")
train_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test, "TF-IDF")
train_evaluate(X_train_bow, X_test_bow, y_train, y_test, "BoW (Frequency)")


✅ One Hot Encoding Accuracy: 0.67

✅ TF-IDF Accuracy: 0.33

✅ BoW (Frequency) Accuracy: 0.33


#Second

In [ ]:
!pip install --upgrade numpy
!pip install --force-reinstall --no-cache-dir gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 182.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 145.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 175.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 145.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 182.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 7.1.0
    Uninstalling smart-open-7.1.0:
      Successfully un

In [ ]:
import re
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# 1. Sentences and Labels
positive_sentences = [
    "The unclean room was finally cleaned and looked beautiful.",
    "Even the unclean utensils were washed and organized nicely.",
    "He saw the unclean water but fixed the source quickly.",
    "The unclean city streets were transformed into green parks.",
    "Though it was unclean, the area was sanitized with care."
]

negative_sentences = [
    "The unclean bathroom smelled terrible and was unusable.",
    "Unclean food caused food poisoning at the party.",
    "She disliked the unclean environment of the hostel.",
    "The unclean river emitted a foul odor.",
    "They were disgusted by the unclean conditions in the hotel."
]

sentences = positive_sentences + negative_sentences
labels = [1]*5 + [0]*5  # 1 = Positive, 0 = Negative


In [ ]:
# 2. Preprocessing and Tokenization
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_sentences = [preprocess(s) for s in sentences]
tokenized_sentences = [s.split() for s in cleaned_sentences]
print("\n--- Tokenized Sentences ---")
for i, tokens in enumerate(tokenized_sentences):
    print(f"{i+1}. {tokens}")


--- Tokenized Sentences ---
1. ['the', 'unclean', 'room', 'was', 'finally', 'cleaned', 'and', 'looked', 'beautiful']
2. ['even', 'the', 'unclean', 'utensils', 'were', 'washed', 'and', 'organized', 'nicely']
3. ['he', 'saw', 'the', 'unclean', 'water', 'but', 'fixed', 'the', 'source', 'quickly']
4. ['the', 'unclean', 'city', 'streets', 'were', 'transformed', 'into', 'green', 'parks']
5. ['though', 'it', 'was', 'unclean', 'the', 'area', 'was', 'sanitized', 'with', 'care']
6. ['the', 'unclean', 'bathroom', 'smelled', 'terrible', 'and', 'was', 'unusable']
7. ['unclean', 'food', 'caused', 'food', 'poisoning', 'at', 'the', 'party']
8. ['she', 'disliked', 'the', 'unclean', 'environment', 'of', 'the', 'hostel']
9. ['the', 'unclean', 'river', 'emitted', 'a', 'foul', 'odor']
10. ['they', 'were', 'disgusted', 'by', 'the', 'unclean', 'conditions', 'in', 'the', 'hotel']


In [ ]:
# 3. Train Word2Vec Model (CBOW = sg=0)
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=50, window=3, min_count=1, sg=0, epochs=300)

In [ ]:
# 4. Get Sentence Vectors (average of word vectors)
def get_sentence_vector(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(50)

X_w2v = np.array([get_sentence_vector(tokens) for tokens in tokenized_sentences])
print("\n--- Word2Vec Sentence Embeddings ---")
print(pd.DataFrame(X_w2v).round(3))



--- Word2Vec Sentence Embeddings ---
      0      1      2      3      4      5      6      7      8      9   ...  \
0  0.002  0.009 -0.040  0.177  0.014  0.001  0.252  0.196 -0.273 -0.013  ...   
1  0.007  0.008 -0.036  0.169  0.008  0.001  0.233  0.186 -0.262 -0.015  ...   
2 -0.003  0.014 -0.042  0.189  0.010 -0.002  0.262  0.203 -0.290 -0.019  ...   
3  0.001  0.006 -0.032  0.173 -0.002  0.001  0.245  0.185 -0.279 -0.018  ...   
4 -0.004  0.003 -0.042  0.195  0.021 -0.003  0.261  0.204 -0.298 -0.011  ...   
5 -0.001  0.008 -0.044  0.187  0.016  0.003  0.253  0.189 -0.284 -0.012  ...   
6  0.007  0.014 -0.035  0.158 -0.002 -0.000  0.231  0.180 -0.250 -0.015  ...   
7  0.006  0.011 -0.035  0.183 -0.000 -0.002  0.234  0.182 -0.269 -0.019  ...   
8  0.001  0.006 -0.041  0.168  0.006  0.014  0.232  0.172 -0.245 -0.017  ...   
9  0.002  0.011 -0.034  0.189  0.013 -0.000  0.257  0.198 -0.289 -0.012  ...   

      40     41     42     43     44     45     46     47     48     49  
0  0.14

In [ ]:
# 5. Train/Test Split
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X_w2v, y, test_size=0.3, random_state=42)

In [ ]:
# 6. Train SVM
model = SVC(kernel='linear')
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [ ]:
# 7. Evaluate
acc = accuracy_score(y_test, preds)
print(f"\n✅ Word2Vec Accuracy: {acc:.2f}")


✅ Word2Vec Accuracy: 0.33


#Third

In [ ]:
import numpy as np
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# 1. Sentences and Labels
positive_sentences = [
    "The unclean room was finally cleaned!!! and looked beautiful.",
    "Even the unclean utensils were washed and organized nicely.",
    "He saw the unclean water but fixed the source quickly.",
    "The unclean city streets were transformed into green parks.",
    "Though it was unclean, the area was sanitized with care."
]

negative_sentences = [
    "The unclean bathroom smelled terrible and was unusable.",
    "Unclean food caused food poisoning at the party.",
    "She disliked the unclean environment of the hostel.",
    "The unclean river emitted a foul odor.",
    "They were disgusted by the unclean conditions in the hotel."
]

sentences = positive_sentences + negative_sentences
labels = [1]*5 + [0]*5  # 1 = Positive, 0 = Negative

In [ ]:
# 2. Preprocessing Function (Lowercase, Remove Punctuation, etc.)
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

cleaned_sentences = [preprocess(s) for s in sentences]
print("\n--- Cleaned Sentences ---")
for i, s in enumerate(cleaned_sentences):
    print(f"{i+1}. {s}")


--- Cleaned Sentences ---
1. the unclean room was finally cleaned and looked beautiful
2. even the unclean utensils were washed and organized nicely
3. he saw the unclean water but fixed the source quickly
4. the unclean city streets were transformed into green parks
5. though it was unclean the area was sanitized with care
6. the unclean bathroom smelled terrible and was unusable
7. unclean food caused food poisoning at the party
8. she disliked the unclean environment of the hostel
9. the unclean river emitted a foul odor
10. they were disgusted by the unclean conditions in the hotel


In [ ]:
# 3. Tokenize the Sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_sentences)
X = tokenizer.texts_to_sequences(cleaned_sentences)

# Pad sequences to make them of equal length
X = pad_sequences(X, padding='post')

In [ ]:
# 4. Train/Test Split
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 5. Build the RNN Model with Embedding Layer
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,  # Vocabulary size
                    output_dim=100,  # Embedding dimension
                    input_length=X_train.shape[1]))  # Length of input sequences
model.add(SimpleRNN(10))  # RNN layer with 10 units
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# 6. Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# 7. Train the Model
model.fit(X_train, y_train, epochs=5, batch_size=2)

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.3714 - loss: 0.7201  
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.5591 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.4942
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.4071
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.3379


In [ ]:
# 8. Evaluate the Model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to 0/1


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step


In [ ]:
# 9. Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Model Accuracy: {acc:.2f}")


✅ Model Accuracy: 0.67


#Fourth

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Example Sentences
sentences = [
    "The unclean room was cleaned beautifully.",
    "She dislikes the unclean environment.",
    "The river is unclean and polluted in each area."
]

# 2. Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)  # Fit tokenizer on the sentences

# 3. Tokenizing the sentences (Converting words to integers)
X = tokenizer.texts_to_sequences(sentences)

print("\n--- Tokenized Sentences ---")
for i, seq in enumerate(X):
    print(f"Sentence {i+1}: {seq}")

# Output the vocabulary (word -> index mapping)
print("\n--- Word Index Mapping ---")
print(tokenizer.word_index)

# 4. Padding the sequences
X_padded = pad_sequences(X, padding='post')

print("\n--- Padded Sequences ---")
for i, seq in enumerate(X_padded):
    print(f"Sentence {i+1} (Padded): {seq}")



--- Tokenized Sentences ---
Sentence 1: [1, 2, 3, 4, 5, 6]
Sentence 2: [7, 8, 1, 2, 9]
Sentence 3: [1, 10, 11, 2, 12, 13, 14, 15, 16]

--- Word Index Mapping ---
{'the': 1, 'unclean': 2, 'room': 3, 'was': 4, 'cleaned': 5, 'beautifully': 6, 'she': 7, 'dislikes': 8, 'environment': 9, 'river': 10, 'is': 11, 'and': 12, 'polluted': 13, 'in': 14, 'each': 15, 'area': 16}

--- Padded Sequences ---
Sentence 1 (Padded): [1 2 3 4 5 6 0 0 0]
Sentence 2 (Padded): [7 8 1 2 9 0 0 0 0]
Sentence 3 (Padded): [ 1 10 11  2 12 13 14 15 16]


#Sample Recommendation System

In [ ]:
import pandas as pd

data = {
    'title': [
        "The Silent River", "Alien Invasion", "Love in Paris", "Space Explorers", "Haunted Memories",
        "Warrior's Path", "Secrets of the Past", "The Hacker's Game", "Forest of Dreams", "Future Code",
        "The Time Loop", "Crimson Dagger", "Ocean Depths", "Cyber Love", "Lost Kingdom",
        "Comedy Central", "Desert Winds", "Broken Melody", "City Shadows", "Dream Catcher"
    ],
    'genre': [
        "Drama", "Sci-Fi", "Romance", "Sci-Fi", "Horror",
        "Action", "Mystery", "Thriller", "Fantasy", "Sci-Fi",
        "Sci-Fi", "Action", "Adventure", "Romance", "Fantasy",
        "Comedy", "Adventure", "Drama", "Thriller", "Mystery"
    ],
    'plot': [
        "A man returns to his hometown and uncovers forgotten memories.",
        "Aliens attack Earth and humans fight to survive.",
        "Two strangers fall in love during a trip to Paris.",
        "Astronauts discover a new planet beyond Mars.",
        "A woman hears voices in a haunted mansion.",
        "A warrior seeks revenge for his lost family.",
        "An old diary reveals shocking secrets from the past.",
        "A hacker gets trapped in his own virtual creation.",
        "A girl finds a magical portal in the forest.",
        "In a tech-driven future, AI begins to rebel.",
        "A scientist stuck in a time loop tries to fix the past.",
        "A detective tracks a killer with a crimson dagger.",
        "Divers find a hidden civilization under the sea.",
        "Two lovers connect through digital devices.",
        "A prince must reclaim his lost throne.",
        "Friends get into hilarious mischief at a comedy club.",
        "A tribe survives harsh conditions in the desert.",
        "A musician searches for meaning after a tragic loss.",
        "A journalist uncovers secrets in a corrupted city.",
        "Teenagers enter dreams to fight their fears."
    ],
    'director_actors': [
        "John Doe, Emily Stone", "Sarah Lynn, Tom Cruise", "Alex Wright, Julia Roberts", "Nolan Pierce, Chris Pratt",
        "Lisa Holmes, Kate Winslet", "Dan Moore, Jason Momoa", "Mira Lee, Rachel McAdams", "Tom Hanks, Ava Green",
        "Jackie Chan, Emma Watson", "Rob Lowe, Emma Stone", "Mark Ruffalo, Keira Knightley", "David Fincher, Brad Pitt",
        "Sophie Turner, Leonardo DiCaprio", "Emma Thompson, Ryan Gosling", "Chris Evans, Natalie Portman",
        "Ben Stiller, Adam Sandler", "Angelina Jolie, Rami Malek", "Lady Gaga, Hugh Jackman", "Morgan Freeman, Anne Hathaway",
        "Millie Bobby Brown, Noah Schnapp"
    ]
}

df = pd.DataFrame(data)
df

,title,genre,plot,director_actors
0,The Silent River,Drama,A man returns to his hometown and uncovers for...,"John Doe, Emily Stone"
1,Alien Invasion,Sci-Fi,Aliens attack Earth and humans fight to survive.,"Sarah Lynn, Tom Cruise"
2,Love in Paris,Romance,Two strangers fall in love during a trip to Pa...,"Alex Wright, Julia Roberts"
3,Space Explorers,Sci-Fi,Astronauts discover a new planet beyond Mars.,"Nolan Pierce, Chris Pratt"
4,Haunted Memories,Horror,A woman hears voices in a haunted mansion.,"Lisa Holmes, Kate Winslet"
5,Warrior's Path,Action,A warrior seeks revenge for his lost family.,"Dan Moore, Jason Momoa"
6,Secrets of the Past,Mystery,An old diary reveals shocking secrets from the...,"Mira Lee, Rachel McAdams"
7,The Hacker's Game,Thriller,A hacker gets trapped in his own virtual creat...,"Tom Hanks, Ava Green"
8,Forest of Dreams,Fantasy,A girl finds a magical portal in the forest.,"Jackie Chan, Emma Watson"
9,Future Code,Sci-Fi,"In a tech-driven future, AI begins to rebel.","Rob Lowe, Emma Stone"


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Combine all text columns
df['combined_text'] = df['genre'] + " " + df['plot'] + " " + df['director_actors']
df['processed'] = df['combined_text'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,title,genre,plot,director_actors,combined_text,processed
0,The Silent River,Drama,A man returns to his hometown and uncovers for...,"John Doe, Emily Stone",Drama A man returns to his hometown and uncove...,drama man return hometown uncov forgotten memo...
1,Alien Invasion,Sci-Fi,Aliens attack Earth and humans fight to survive.,"Sarah Lynn, Tom Cruise",Sci-Fi Aliens attack Earth and humans fight to...,scifi alien attack earth human fight surviv sa...
2,Love in Paris,Romance,Two strangers fall in love during a trip to Pa...,"Alex Wright, Julia Roberts",Romance Two strangers fall in love during a tr...,romanc two stranger fall love trip pari alex w...
3,Space Explorers,Sci-Fi,Astronauts discover a new planet beyond Mars.,"Nolan Pierce, Chris Pratt",Sci-Fi Astronauts discover a new planet beyond...,scifi astronaut discov new planet beyond mar n...
4,Haunted Memories,Horror,A woman hears voices in a haunted mansion.,"Lisa Holmes, Kate Winslet",Horror A woman hears voices in a haunted mansi...,horror woman hear voic haunt mansion lisa holm...
5,Warrior's Path,Action,A warrior seeks revenge for his lost family.,"Dan Moore, Jason Momoa",Action A warrior seeks revenge for his lost fa...,action warrior seek reveng lost famili dan moo...
6,Secrets of the Past,Mystery,An old diary reveals shocking secrets from the...,"Mira Lee, Rachel McAdams",Mystery An old diary reveals shocking secrets ...,mysteri old diari reveal shock secret past mir...
7,The Hacker's Game,Thriller,A hacker gets trapped in his own virtual creat...,"Tom Hanks, Ava Green",Thriller A hacker gets trapped in his own virt...,thriller hacker get trap virtual creation tom ...
8,Forest of Dreams,Fantasy,A girl finds a magical portal in the forest.,"Jackie Chan, Emma Watson",Fantasy A girl finds a magical portal in the f...,fantasi girl find magic portal forest jacki ch...
9,Future Code,Sci-Fi,"In a tech-driven future, AI begins to rebel.","Rob Lowe, Emma Stone","Sci-Fi In a tech-driven future, AI begins to r...",scifi techdriven futur ai begin rebel rob low ...


In [ ]:
print(df['combined_text'][0])
print(df['processed'][0])

Drama A man returns to his hometown and uncovers forgotten memories. John Doe, Emily Stone
drama man return hometown uncov forgotten memori john doe emili stone


In [ ]:
print(df['combined_text'][5])
print(df['processed'][5])

Action A warrior seeks revenge for his lost family. Dan Moore, Jason Momoa
action warrior seek reveng lost famili dan moor jason momoa


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['processed'])

# Show TF-IDF vector for first movie
print("TF-IDF vector for:", df['title'][0])
print(tfidf_matrix[0].toarray())

TF-IDF vector for: The Silent River
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.31131651 0.27365198
  0.         0.         0.31131651 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.31131651 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.31131651 0.         0.
  0.         0.         0.         0.         0.31131651 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        

In [ ]:
tfidf_matrix[0].toarray().shape

(1, 184)

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Tokenize for Word2Vec
tokenized = df['processed'].apply(lambda x: x.split())
model_w2v = Word2Vec(tokenized, vector_size=100, window=5, min_count=1, workers=4)

def get_avg_vector(words, model):
    valid_words = [w for w in words if w in model.wv]
    if not valid_words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[valid_words], axis=0)

# Apply for all
w2v_vectors = np.array([get_avg_vector(words, model_w2v) for words in tokenized])

# Show Word2Vec vector
print("Word2Vec vector for:", df['title'][0])
print(w2v_vectors[0])

Word2Vec vector for: The Silent River
[-3.3739445e-05  2.6075786e-03  1.5648683e-04  2.8222130e-04
  1.1263975e-03 -1.3805124e-03  5.7964669e-05  2.3657011e-03
  2.4490631e-03  1.3327668e-03 -1.0642976e-03  4.5006213e-04
  1.4781873e-03  8.1299600e-04  3.0081607e-03 -2.9666006e-04
 -1.2521633e-03  9.5451024e-04  1.5424583e-03  6.3802360e-04
 -1.7164796e-04  1.2884001e-05 -6.0166628e-04  1.0123043e-03
 -5.0504785e-04  2.1072591e-03 -5.5997074e-04 -1.2554459e-03
  4.4413824e-03  7.4270146e-04  5.9308769e-04 -2.1500478e-03
 -1.1702656e-03  2.1319874e-03 -1.8855922e-03 -3.2735858e-04
  1.2311161e-03 -2.1405129e-03  1.3628964e-03 -2.1349073e-03
 -2.4220334e-04  6.2747567e-04  4.0217338e-04 -2.2907197e-03
 -3.8146516e-03  6.6526751e-05 -1.5009635e-03 -8.8208145e-04
  1.5917864e-03  9.1721700e-04 -3.1184768e-03 -2.3731560e-04
 -1.3497178e-03 -5.4171577e-04  1.9906035e-03  2.6070487e-03
  1.8991376e-03 -2.7686320e-04  1.4657431e-03 -1.3278122e-03
  9.2533644e-04  6.9781468e-04 -1.4420395e-03 -

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(user_input, vector_type='tfidf', top_n=3):
    input_processed = preprocess_text(user_input)

    if vector_type == 'tfidf':
        input_vec = tfidf.transform([input_processed])
        sims = cosine_similarity(input_vec, tfidf_matrix)[0]
    else:
        input_vec = get_avg_vector(input_processed.split(), model_w2v).reshape(1, -1)
        sims = cosine_similarity(input_vec, w2v_vectors)[0]

    top_idx = sims.argsort()[-top_n:][::-1]
    for idx in top_idx:
        print(f"{df['title'][idx]} (Score: {sims[idx]:.2f})")

# Try it out
user_input = "Time travel scientist"
print("\n🔍 Recommendations using TF-IDF:")
recommend_movies(user_input, vector_type='tfidf')

print("\n🔍 Recommendations using Word2Vec:")
recommend_movies(user_input, vector_type='w2v')


🔍 Recommendations using TF-IDF:
The Time Loop (Score: 0.42)
Dream Catcher (Score: 0.00)
Forest of Dreams (Score: 0.00)

🔍 Recommendations using Word2Vec:
The Time Loop (Score: 0.25)
Warrior's Path (Score: 0.15)
Space Explorers (Score: 0.15)


#Predicting Next Words

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
corpus = [
    "the sun rises in the east",
    "the moon shines at night",
    "the stars twinkle in the sky",
    "the sun sets in the west",
    "the sky is blue and vast",
    "the night is dark and quiet",
    "the stars shine brightly at night",
    "the children play in the park",
    "the dog barked loudly at strangers",
    "the cat sleeps on the couch",
    "the boy reads a book every day",
    "the girl sings a beautiful song",
    "the bird flies in the sky",
    "the wind blows gently at night",
    "the rain falls on the roof",
    "the flowers bloom in spring",
    "the trees sway in the wind",
    "the baby smiles at her mother",
    "the teacher writes on the board",
    "the students listen to the lesson",
]

In [ ]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1  # +1 for padding
print("Total words:", total_words)

Total words: 68


In [ ]:
# Create input sequences (1 to n)
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] # [[1, 2, 3]] [0] returns [1,2,3]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

In [ ]:
# Pad sequences
max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')


In [ ]:
# Split predictors and label
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

In [ ]:
# Define LSTM model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=10, input_length=max_seq_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0061 - loss: 4.2194   
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0865 - loss: 4.2143
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0800 - loss: 4.2107
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1081 - loss: 4.2060
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0998 - loss: 4.2009
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1269 - loss: 4.1920
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0987 - loss: 4.1792
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1081 - loss: 4.1664
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1112 - loss: 4.1420
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1206 - loss: 4.1148
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1060 - loss: 4.0686
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0894 -

In [ ]:
# Function to predict next word(s)
def predict_next_words(seed_text, n_words):
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:
# Test prediction
seed = "the sun"
next_words = 3
print("Predicted:", predict_next_words(seed, next_words))

Predicted: the sun sets in the


In [ ]:
# Test prediction
seed = "The children"
next_words = 5
print("Predicted:", predict_next_words(seed, next_words))

Predicted: The children sets in the park board
